<a href="https://colab.research.google.com/github/atsiolis/sms_text_classification/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

sms_train = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
sms_test = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])



In [ ]:
# Convert labels to binary
sms_train['label'] = sms_train['label'].map({'ham': 0, 'spam': 1})
sms_test['label'] = sms_test['label'].map({'ham': 0, 'spam': 1})

X_train, y_train = sms_train['message'], sms_train['label']
X_test, y_test = sms_test['message'], sms_test['label']

In [ ]:
# Plot the data set
plt.bar('ham', sms_train[sms_train['label'] == 0].shape[0])
plt.bar('spam', sms_train[sms_train['label'] == 1].shape[0])
plt.show()

plt.bar('ham', sms_test[sms_test['label'] == 0].shape[0])
plt.bar('spam', sms_test[sms_test['label'] == 1].shape[0])
plt.show()

In [ ]:
# Tokenization and padding
max_vocab_size = 20000  # Increase vocabulary size
max_length = 150  # Increase sequence length
embedding_dim = 32  # Increase embedding dimension

tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_length, padding='post')
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_length, padding='post')

In [ ]:
# Build the model
model = keras.Sequential([
    keras.layers.Embedding(max_vocab_size, 16),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(X_train_seq, y_train, epochs=20, validation_data=(X_test_seq, y_test), verbose=1)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  seq = pad_sequences(tokenizer.texts_to_sequences([pred_text]), maxlen=max_length, padding='post')
  prob = model.predict(seq)[0][0]
  label = "spam" if prob > 0.5 else "ham"
  return [prob, label]


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
